
# 🧠 کارگاه تحلیل نظرات فارسی دیجی‌کالا (Persian NLP 2025)

در این نوت‌بوک، گام‌به‌گام روی دیتاست نظرات دیجی‌کالا کار می‌کنیم و این مراحل را انجام می‌دهیم:

1. آشنایی با دیتاست و چالش‌های متون فارسی  
2. پاک‌سازی و نرمال‌سازی متن فارسی  
3. روش کلاسیک: **TF-IDF + مدل‌های یادگیری ماشین**  
4. روش مدرن: **Embedding با BGE-M3**  
5. مقایسه عملکرد TF-IDF و Embedding  
6. نمایش فضای معنایی با UMAP  
7. استفاده‌ی مقدماتی از LLMها برای تحلیل احساس و خلاصه‌سازی  
8. تمرین پایانی: تحلیل کامل نظرات یک محصول



## 1️⃣ نصب کتابخانه‌های موردنیاز

در این بخش، کتابخانه‌های اصلی مورد استفاده را نصب می‌کنیم:

- `pandas`, `numpy` برای کار با داده‌ها  
- `scikit-learn` برای TF-IDF و مدل‌های یادگیری ماشین  
- `matplotlib` برای رسم نمودارها  
- `sentence-transformers` برای ساخت embedding (مدل BGE-M3)  
- `umap-learn` برای کاهش بُعد و نمایش بصری embeddingها  

> نکته: نصب پکیج‌ها در Colab ممکن است چند دقیقه زمان ببرد.


In [ ]:
# نصب پکیج‌های اصلی (در صورت نیاز)
!pip install -q pandas numpy scikit-learn matplotlib
!pip install -q sentence-transformers
!pip install -q umap-learn

# کتابخانه‌های اختیاری (در صورت نیاز uncomment کنید)
# !pip install -q bertopic[all]
# !pip install -q faiss-cpu



## 2️⃣ بارگذاری دیتاست نظرات دیجی‌کالا

در این کارگاه از دیتاست آماده‌ی نظرات فارسی دیجی‌کالا استفاده می‌کنیم.

- اگر دیتاست را از Kaggle دانلود کرده‌اید، آن را در مسیر مناسب قرار دهید.  
- شکل دیتاست فرضی:  
  - `Text` : متن کامل نظر مشتری  
  - `Score`: امتیاز عددی (۱ تا ۵)  
  - `Suggestion`: آیا کاربر خرید را پیشنهاد می‌کند یا نه (اختیاری)


In [ ]:
import pandas as pd

# اگر فایل را در Colab آپلود کرده‌اید، نام فایل را اینجا بگذارید
DATA_PATH = "data.csv"  # یا نام فایلی که از Kaggle گرفته‌اید

df = pd.read_csv(DATA_PATH)
df.head()



### نگاهی سریع به داده‌ها

توزیع امتیازها را بررسی می‌کنیم تا ببینیم داده‌ها متوازن هستند یا نه.


In [ ]:
df.info()

In [ ]:
df['Score'].value_counts().sort_index()


## 3️⃣ تعریف برچسب احساس (Sentiment Label)

برای ساده‌سازی مسئله، امتیازها را به سه کلاس تبدیل می‌کنیم:

- **۱ و ۲ → منفی (negative)**  
- **۳ → خنثی (neutral)** (در صورت کم بودن، می‌توانیم حذف کنیم)  
- **۴ و ۵ → مثبت (positive)**  

در این نوت‌بوک برای سادگی، فقط دو کلاس مثبت/منفی را نگه می‌داریم و کلاس خنثی را حذف می‌کنیم.


In [ ]:
def score_to_label(score):
    if score <= 2:
        return "negative"
    elif score >= 4:
        return "positive"
    else:
        return "neutral"

df['label'] = df['Score'].apply(score_to_label)

# حذف نظرات خنثی (در صورت کم بودن)
df = df[df['label'] != 'neutral'].reset_index(drop=True)

df['label'].value_counts()


## 4️⃣ پاک‌سازی و نرمال‌سازی متن فارسی

در متون فارسی مشکلات رایجی داریم:

- مخلوط شدن حروف عربی و فارسی (`ي` / `ی`، `ك` / `ک`)  
- وجود اموجی‌ها، URLها، @mentionها و …  
- علائم نگارشی اضافی  
- فاصله‌های تکراری  
- اعداد فارسی و انگلیسی  

در این بخش یک تابع `clean_persian_text` پیاده‌سازی می‌کنیم که کارهای زیر را انجام می‌دهد:

1. یکنواخت‌سازی حروف (`ي`→`ی` و `ك`→`ک`)  
2. حذف URL، ایمیل، @mention و هشتگ  
3. حذف اموجی‌ها و کاراکترهای غیرحرفی غیرضروری  
4. حذف علائم نگارشی و تبدیل آنها به فاصله  
5. حذف فاصله‌های تکراری و strip کردن متن  
6. تبدیل همه چیز به حروف کوچک (در فارسی تأثیر زیادی ندارد اما برای یکدستی مفید است)

> نکته: اگر خواستید می‌توانید روی این متن‌ها، Stemming یا Lemmatization فارسی (مثلاً با `hazm` یا `parsivar`) نیز انجام دهید.


In [ ]:
import re

# الگوی اموجی‌ها (ساده)
emoji_pattern = re.compile("""["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
"]+""", flags=re.UNICODE)

def normalize_arabic_chars(text: str) -> str:
    # ي/ی و ك/ک
    text = text.replace('ي', 'ی').replace('ى', 'ی')
    text = text.replace('ك', 'ک')
    return text

def convert_persian_digits(text: str) -> str:
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    trans_table = str.maketrans(''.join(persian_digits), ''.join(english_digits))
    return text.translate(trans_table)

def clean_persian_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = normalize_arabic_chars(text)
    text = convert_persian_digits(text)

    # حذف URLها
    text = re.sub(r'http\S+|www\.\S+', ' ', text)

    # حذف ایمیل
    text = re.sub(r'\S+@\S+', ' ', text)

    # حذف @mention و هشتگ
    text = re.sub(r'[@#]\S+', ' ', text)

    # حذف اموجی‌ها
    text = emoji_pattern.sub(' ', text)

    # حذف هرچیزی به جز حروف و اعداد و فاصله
    text = re.sub(r'[^۰-۹0-9آ-یئءچژگۀ۱۲۳۴۵۶۷۸۹\s]', ' ', text)

    # تبدیل چند فاصله به یکی
    text = re.sub(r'\s+', ' ', text).strip()

    # کوچک‌سازی (در فارسی حساسیت کمتری دارد)
    text = text.lower()

    return text

df['clean_text'] = df['Text'].astype(str).apply(clean_persian_text)
df[['Text', 'clean_text']].head(10)


## 5️⃣ روش کلاسیک: TF-IDF + مدل‌های یادگیری ماشین

در این بخش:

1. متن‌های `clean_text` را با `TfidfVectorizer` به بردار تبدیل می‌کنیم.  
2. داده‌ها را به train/test تقسیم می‌کنیم.  
3. چند مدل را روی TF-IDF آموزش می‌دهیم:
   - Logistic Regression  
   - Linear SVM  
   - Random Forest  
4. معیارهای ارزیابی (Accuracy، Precision، Recall، F1) و Confusion Matrix را بررسی می‌کنیم.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# ساخت TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.9
)

X_tfidf = tfidf_vectorizer.fit_transform(df['clean_text'])
y = df['label']

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

X_tfidf.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

tfidf_results = []

def evaluate_model_tfidf(name, model):
    model.fit(X_train_tfidf, y_train_tfidf)
    preds = model.predict(X_test_tfidf)

    acc = accuracy_score(y_test_tfidf, preds)
    p, r, f, _ = precision_recall_fscore_support(
        y_test_tfidf, preds, average="weighted", zero_division=0
    )

    tfidf_results.append([name, acc, p, r, f])
    print(f"===== {name} (TF-IDF) =====")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {p:.4f} | Recall: {r:.4f} | F1: {f:.4f}")
    print()
    print(classification_report(y_test_tfidf, preds))
    
    # Confusion Matrix
    labels = sorted(y.unique())
    cm = confusion_matrix(y_test_tfidf, preds, labels=labels)
    fig, ax = plt.subplots(figsize=(4,4))
    im = ax.imshow(cm)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    for i in range(len(labels)):
        for j in range(len(labels)):
            ax.text(j, i, cm[i, j], ha="center", va="center", color="w")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(f"Confusion Matrix - {name} (TF-IDF)")
    plt.show()


# اجرای مدل‌ها
lr_t = LogisticRegression(max_iter=3000)
evaluate_model_tfidf("Logistic Regression", lr_t)

svm_t = LinearSVC()
evaluate_model_tfidf("Linear SVM", svm_t)

rf_t = RandomForestClassifier(n_estimators=300)
evaluate_model_tfidf("Random Forest", rf_t)


## 6️⃣ روش مدرن: Embedding با BGE-M3

در روش‌های مدرن NLP به جای شمارش کلمات، هر جمله/نظر را به یک **بردار معنایی** در فضای با بُعد بالا تبدیل می‌کنیم.

در این بخش از مدل **BGE-M3** استفاده می‌کنیم که:

- چندزبانه است (از جمله فارسی)  
- برای بازیابی معنایی (Semantic Search) و Classification عالی عمل می‌کند  
- هر متن را به یک بردار ۱۰۲۴بُعدی تبدیل می‌کند

مراحل:

1. لود کردن مدل `BAAI/bge-m3` با `sentence-transformers`  
2. تبدیل همه‌ی `clean_text`ها به embedding  
3. آموزش همان مدل‌های قبلی (LR، SVM، RF، XGBoost، LightGBM، KNN، MLP) روی embeddingها  
4. مقایسه نتایج با TF-IDF


In [ ]:
from sentence_transformers import SentenceTransformer

print("Loading BGE-M3 model...")
emb_model = SentenceTransformer("BAAI/bge-m3")

embeddings = emb_model.encode(
    df['clean_text'].tolist(),
    batch_size=32,
    show_progress_bar=True
)

embeddings.shape

In [ ]:
from sklearn.model_selection import train_test_split

X = embeddings
y = df['label']

X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import lightgbm as lgb

emb_results = []

def evaluate_model_emb(name, model, encoded=False):
    if encoded:
        le = LabelEncoder()
        y_enc = le.fit_transform(y)
        X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(
            X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
        )
        model.fit(X_train_e, y_train_e)
        preds = model.predict(X_test_e)
        true_y = y_test_e
    else:
        model.fit(X_train_emb, y_train_emb)
        preds = model.predict(X_test_emb)
        true_y = y_test_emb

    acc = accuracy_score(true_y, preds)
    p, r, f, _ = precision_recall_fscore_support(
        true_y, preds, average="weighted", zero_division=0
    )

    emb_results.append([name, acc, p, r, f])

    print(f"===== {name} (Embedding) =====")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {p:.4f} | Recall: {r:.4f} | F1: {f:.4f}")
    print()


# مدل‌های کلاسیک روی Embedding
evaluate_model_emb("Logistic Regression", LogisticRegression(max_iter=3000))
evaluate_model_emb("Linear SVM", LinearSVC())
evaluate_model_emb("Random Forest", RandomForestClassifier(n_estimators=300))
evaluate_model_emb("KNN", KNeighborsClassifier(n_neighbors=7))
evaluate_model_emb("MLP Neural Network", MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500))

# مدل‌های گرادیان‌بوسینگ با لیبل عددی
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric='mlogloss'
)
evaluate_model_emb("XGBoost", xgb, encoded=True)

lgbm = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.1)
evaluate_model_emb("LightGBM", lgbm, encoded=True)


## 7️⃣ مقایسه‌ی TF-IDF و Embedding

در این بخش نتایج مدل‌ها را در دو حالت مقایسه می‌کنیم:

- نمایش جدولی  
- نمایش نموداری (نمودار میله‌ای F1-score)

هدف: نشان دهیم که Embedding معمولاً:
- دقت و F1 بالاتری می‌دهد  
- برای کارهای بعدی مثل Semantic Search و RAG قابل استفاده است  
در حالی که TF-IDF بیشتر برای نسل قدیمی‌تر NLP مناسب است.


In [ ]:
import pandas as pd

tfidf_df = pd.DataFrame(tfidf_results, columns=["Model","Accuracy","Precision","Recall","F1"])
tfidf_df["Representation"] = "TF-IDF"

emb_df = pd.DataFrame(emb_results, columns=["Model","Accuracy","Precision","Recall","F1"])
emb_df["Representation"] = "Embedding"

compare_df = pd.concat([tfidf_df, emb_df], ignore_index=True)
compare_df

In [ ]:
compare_pivot = compare_df.pivot(index="Model", columns="Representation", values="F1")
compare_pivot

In [ ]:
compare_pivot.plot(kind="bar", figsize=(10,6))
plt.title("مقایسه F1 مدل‌ها: TF-IDF vs Embedding")
plt.ylabel("F1-score")
plt.xticks(rotation=45)
plt.show()


## 8️⃣ نمایش بصری فضای معنایی با UMAP

برای درک بهتر Embeddingها، آنها را با UMAP از ۱۰۲۴ بُعد به ۲ بُعد کاهش می‌دهیم و:

- هر نقطه = یک نظر مشتری  
- رنگ = برچسب احساس (مثبت / منفی)

این Visualization کمک می‌کند:

- خوشه‌بندی معنایی نظرات را ببینیم  
- جدایی کلاس‌ها را در فضای embedding مشاهده کنیم


In [ ]:
import umap.umap_ as umap

reducer = umap.UMAP(
    n_components=2,
    n_neighbors=30,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

umap_2d = reducer.fit_transform(embeddings)

df['umap_x'] = umap_2d[:,0]
df['umap_y'] = umap_2d[:,1]

plt.figure(figsize=(8,6))
for label, color in zip(['positive','negative'], ['tab:blue','tab:orange']):
    subset = df[df['label'] == label]
    plt.scatter(subset['umap_x'], subset['umap_y'], s=10, label=label)
plt.legend()
plt.title("UMAP Visualization of Persian Comment Embeddings")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.show()


## 9️⃣ استفاده‌ی مقدماتی از LLMها (خلاصه‌سازی و تحلیل احساس)

در کنار رویکرد کلاسیک (ML + Embedding)، می‌توانیم از LLMها نیز استفاده کنیم:

**کاربردها:**

- خلاصه‌سازی چند نظر مشتری  
- استخراج نکات مثبت و منفی  
- تحلیل احساس به صورت Zero-shot یا Few-shot  
- تولید تحلیل مدیریتی به زبان طبیعی

در Colab می‌توانید بسته به دسترسی خودتان از یکی از این‌ها استفاده کنید:

- مدل‌های `transformers` (مثلاً Qwen، Mistral، LLaMA)  
- APIهای LLM (OpenAI، سایر ارائه‌دهنده‌ها)

در اینجا فقط یک شِمای کلی می‌دهیم که شرکت‌کنندگان کارگاه بتوانند بعداً خودشان تست کنند.


In [ ]:
# نمونه کد نمایشی (Pseudo-code) برای استفاده از یک LLM HuggingFace
# توجه: برای اجرا باید مدل مناسب را جایگزین کنید و منابع لازم (GPU) داشته باشید.

# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# model_name = "Qwen/Qwen2.5-3B-Instruct"  # به عنوان مثال
# pipe = pipeline("text-generation", model=model_name, tokenizer=model_name)

# sample_reviews = "\n".join(df['clean_text'].sample(10).tolist())

# prompt = f"""
# این نظرات مشتریان را بخوان و موارد زیر را به زبان فارسی و خلاصه بنویس:

# - خلاصه کلی احساس مشتریان
# - مهم‌ترین نقاط مثبت
# - مهم‌ترین نقاط منفی
# - سه پیشنهاد کلیدی برای بهبود محصول

# نظرات:
# {sample_reviews}
# """

# out = pipe(prompt, max_new_tokens=300)[0]['generated_text']
# print(out)


## 🔟 تمرین پایانی کارگاه

به عنوان تمرین نهایی، مراحل زیر را برای **یک زیرمجموعه از داده‌ها (مثلاً یک محصول خاص یا یک دسته‌ی خاص)** انجام دهید:

1. **فیلتر کردن داده‌ها**  
   - فقط نظرات مربوط به یک محصول مشخص یا یک دسته را انتخاب کنید.

2. **آموزش یک مدل احساس ساده**  
   - از Embedding (BGE-M3) و Logistic Regression استفاده کنید.  
   - دقت و F1 را گزارش کنید.

3. **Topic Modeling (اختیاری، اگر BERTopic نصب کردید)**  
   - ۳ تا ۵ موضوع اصلی در نظرات را استخراج کنید.  
   - برای هر Topic چند نمونه نظر نشان دهید.

4. **Semantic Search (اختیاری)**  
   - یک سؤال مثل «مهم‌ترین دلیل نارضایتی مشتریان چیست؟» تعریف کنید.  
   - نزدیک‌ترین ۱۰ نظر را با فاصله‌ی Cosine روی embeddingها پیدا کنید.

5. **تحلیل مدیریتی نهایی**  
   - در قالب ۵–۷ خط، یک گزارش مدیریتی بنویسید که شامل موارد زیر باشد:
     - خلاصه‌ی کلی احساس مشتریان  
     - نکات مثبت اصلی  
     - مشکلات اصلی مطرح شده توسط مشتریان  
     - پیشنهادهای کلیدی برای واحدهای: محصول، پشتیبانی، لجستیک

> پیشنهاد: خروجی تمرین را در یک سلول Markdown مرتب و قابل ارائه بنویسید تا بتوانید آن را در سازمان خود به اشتراک بگذارید.
